In [ ]:
 %%writefile Prac02a.cpp
#include <stdio.h>
#include <omp.h>
#include <time.h>

void bubbleSort(int arr[], int n) {
    int i, j;
    #pragma omp parallel for private(i, j) shared(arr)
    for (i = 0; i < n-1; i++) {
        // Last i elements are already in place, so only iterate till n-i-1
        for (j = 0; j < n-i-1; j++) {
            // Swap if the element found is greater than the next element
            if (arr[j] > arr[j+1]) {
                int temp = arr[j];
                arr[j] = arr[j+1];
                arr[j+1] = temp;
            }
        }
    }
}

int main() {
    int n;
    printf("Enter the number of elements: ");
    scanf("%d", &n);

    int arr[n];
    printf("Enter the elements: ");
    for (int i = 0; i < n; i++) {
        scanf("%d", &arr[i]);
    }

    printf("Unsorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    double start_time = omp_get_wtime();
    #pragma omp parallel
    {
        bubbleSort(arr, n);
    }
    double end_time = omp_get_wtime();

    printf("Sorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    printf("Total time taken by parallel section: %f seconds\n", end_time - start_time);
    printf("Number of threads used: %d\n", omp_get_max_threads());

    return 0;
}


Writing Prac02a.cpp


In [ ]:
!g++ -fopenmp Prac02a.cpp -o Myexe

In [ ]:
!./Myexe


Enter the number of elements: 5
Enter the elements: 10
27
90
0
22
Unsorted array: 
10 27 90 0 22 
Sorted array: 
0 10 22 27 90 
Total time taken by parallel section: 0.000240 seconds
Number of threads used: 2


In [ ]:
 %%writefile Prac02b.cpp
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include <time.h>

void merge(int arr[], int l, int m, int r) {
    int i, j, k;
    int n1 = m - l + 1;
    int n2 = r - m;

    // Create temporary arrays
    int L[n1], R[n2];

    // Copy data to temporary arrays L[] and R[]
    for (i = 0; i < n1; i++)
        L[i] = arr[l + i];
    for (j = 0; j < n2; j++)
        R[j] = arr[m + 1 + j];

    // Merge the temporary arrays back into arr[l..r]
    i = 0; // Initial index of first subarray
    j = 0; // Initial index of second subarray
    k = l; // Initial index of merged subarray
    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    // Copy the remaining elements of L[], if any
    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    // Copy the remaining elements of R[], if any
    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }
}

void mergeSort(int arr[], int l, int r) {
    if (l < r) {
        // Same as (l+r)/2, but avoids overflow for large l and r
        int m = l + (r - l) / 2;

        // Sort first and second halves in parallel
        #pragma omp parallel sections
        {
            #pragma omp section
            {
                mergeSort(arr, l, m);
            }
            #pragma omp section
            {
                mergeSort(arr, m + 1, r);
            }
        }

        merge(arr, l, m, r);
    }
}

int main() {
    int n;
    printf("Enter the number of elements: ");
    scanf("%d", &n);

    int arr[n];
    printf("Enter the elements: ");
    for (int i = 0; i < n; i++) {
        scanf("%d", &arr[i]);
    }

    printf("Unsorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    double start_time = omp_get_wtime();
    // Perform merge sort
    #pragma omp parallel
    {
        #pragma omp single
        mergeSort(arr, 0, n - 1);
    }
    double end_time = omp_get_wtime();

    printf("Sorted array: \n");
    for (int i = 0; i < n; i++) {
        printf("%d ", arr[i]);
    }
    printf("\n");

    printf("Time taken: %f seconds\n", end_time - start_time);
    printf("Number of threads used: %d\n", omp_get_max_threads());

    return 0;
}


Writing Prac02b.cpp


In [ ]:
!g++ -fopenmp Prac02b.cpp -o Myexe

In [ ]:
!./Myexe


Enter the number of elements: 5
Enter the elements: 2 
7
88
55
21
Unsorted array: 
2 7 88 55 21 
Sorted array: 
2 7 21 55 88 
Time taken: 0.000138 seconds
Number of threads used: 2


In [ ]:
# Let's analyze both parallel versions of the code individually and compare them to their sequential counterparts.

# ### Prac02a.cpp - Parallel Bubble Sort

# - **Parallelism**: In the parallel version, the bubble sort algorithm is executed concurrently by multiple threads using OpenMP parallelization.
# - **Performance Improvement**: The parallel version can potentially exhibit better performance compared to the sequential version for large datasets and systems with multiple cores. This is because parallel execution allows multiple threads to simultaneously work on different portions of the array, potentially reducing the overall execution time.
# - **Time Complexity**: Bubble sort has a time complexity of O(n^2) in both its sequential and parallel versions. Parallelizing bubble sort does not change its time complexity; however, it can improve its performance by distributing the workload across multiple threads.

# ### Prac02b.cpp - Parallel Merge Sort

# - **Parallelism**: In the parallel version, the merge sort algorithm is executed concurrently by multiple threads using OpenMP parallelization.
# - **Performance Improvement**: The parallel version can exhibit significantly better performance compared to the sequential version, especially for large datasets. Merge sort has a time complexity of O(n log n), which makes it more efficient than bubble sort. Parallelizing merge sort allows multiple threads to work on sorting different portions of the array simultaneously, resulting in faster sorting times.
# - **Time Complexity**: Merge sort has a time complexity of O(n log n), both in its sequential and parallel versions. Parallelizing merge sort does not change its time complexity; however, it can significantly improve its performance by leveraging multiple threads to divide and conquer the sorting process efficiently.

# In summary, the parallel versions of both bubble sort and merge sort can potentially exhibit better performance compared to their sequential counterparts, especially for large datasets. While parallelization does not change the time complexities of the algorithms themselves, it can distribute the workload across multiple threads, leading to faster execution times, particularly for algorithms like merge sort with more efficient time complexities.

In [ ]:
 %%writefile Prac02afinal.cpp
#include <iostream>
#include <vector>
#include <omp.h>
#include <cstdlib> // for random number generation
#include <ctime>   // for seeding random number generator
using namespace std;

void bubble_sort_odd_even(vector<int>& arr) {
    bool isSorted = false;
    while (!isSorted) {
        isSorted = true;
        #pragma omp parallel for
        for (int i = 0; i < arr.size() - 1; i += 2) {
            if (arr[i] > arr[i + 1]) {
                swap(arr[i], arr[i + 1]);
                isSorted = false;
            }
        }
        #pragma omp parallel for
        for (int i = 1; i < arr.size() - 1; i += 2) {
            if (arr[i] > arr[i + 1]) {
                swap(arr[i], arr[i + 1]);
                isSorted = false;
            }
        }
    }
}

int main() {
    int n;
    cout << "Enter the number of elements: ";
    cin >> n;

    // Uncomment the following section to generate random values for sorting
    // cout << "Enter the number of elements (or uncomment to generate random values): ";
    // cin >> n;
    // srand(time(0)); // Seed random number generator
    // vector<int> arr(n);
    // for (int i = 0; i < n; ++i) {
    //     arr[i] = rand() % 1000; // Generate random numbers between 0 and 999
    // }

    # // vector<int> arr(n);
    # // cout << "Enter " << n << " elements:\n";
    # // for (int i = 0; i < n; ++i) {
    # //     cin >> arr[i];
    # // }

    double start, end;
    // Measure performance of parallel bubble sort using odd-even transposition
    start = omp_get_wtime();
    bubble_sort_odd_even(arr);
    end = omp_get_wtime();

    cout << "Sorted array:";
    for (int i = 0; i < n; ++i) {
        cout << " " << arr[i];
    }
    cout << endl;

    cout << "Parallel bubble sort using odd-even transposition time: " << end - start << " seconds" << endl;

    return 0;
}


Overwriting Prac02afinal.cpp


In [ ]:
!g++ -fopenmp Prac02afinal.cpp -o Myexe

In [ ]:
!./Myexe

Enter the number of elements: 20
Sorted array: 48 114 283 392 394 395 464 466 483 505 545 563 592 691 695 762 800 876 876 917
Parallel bubble sort using odd-even transposition time: 0.000121076 seconds


In [13]:
 %%writefile Prac02afinall.cpp
#include<iostream>
#include<omp.h>
#include<cstdlib> // for random number generation
#include<ctime>   // for seeding random number generator

using namespace std;

void bubble(int array[], int n){
    for (int i = 0; i < n - 1; i++){
        for (int j = 0; j < n - i - 1; j++){
            if (array[j] > array[j + 1]) swap(array[j], array[j + 1]);
        }
    }
}

void pBubble(int array[], int n){
    //Sort odd indexed numbers
    for(int i = 0; i < n; ++i){
        #pragma omp for
        for (int j = 1; j < n; j += 2){
        if (array[j] < array[j-1])
        {
          swap(array[j], array[j - 1]);
        }
    }

    // Synchronize
    #pragma omp barrier

    //Sort even indexed numbers
    #pragma omp for
    for (int j = 2; j < n; j += 2){
      if (array[j] < array[j-1])
      {
        swap(array[j], array[j - 1]);
      }
    }
  }
}

void printArray(int arr[], int n){
    for(int i = 0; i < n; i++) cout << arr[i] << " ";
    cout << "\n";
}

int main(){
    // Set up variables
    int n;
    cout << "Enter the number of elements: ";
    cin >> n;

    int arr[n];
    int brr[n];
    double start_time, end_time;

    // Uncomment the following section to generate random input
    srand(time(0)); // Seed random number generator
    cout << "Randomly generated array:" << endl;
    for (int i = 0; i < n; ++i) {
        arr[i] = rand() % 1000; // Generate random numbers between 0 and 999
        cout << arr[i] << " ";
    }
    cout << endl;

    // User input
    // cout << "Enter the elements: ";
    // for (int i = 0; i < n; i++) {
    //     cin >> arr[i];
    // }

    // Sequential time
    start_time = omp_get_wtime();
    bubble(arr, n);
    end_time = omp_get_wtime();
    cout << "Sequential Bubble Sort took : " << end_time - start_time << " seconds.\n";
    printArray(arr, n);

    // Reset the array
    for(int i = 0; i < n; i++) brr[i] = arr[i];

    // Parallel time
    start_time = omp_get_wtime();
    pBubble(brr, n);
    end_time = omp_get_wtime();
    cout << "Parallel Bubble Sort took : " << end_time - start_time << " seconds.\n";
    printArray(brr, n);
}


Overwriting Prac02afinall.cpp


In [14]:
!g++ -fopenmp Prac02afinall.cpp -o Myexe

In [16]:
!./Myexe

Enter the number of elements: 100
Randomly generated array:
575 581 928 265 936 59 616 422 72 872 681 700 304 274 849 173 808 127 177 629 23 300 836 558 40 697 966 75 201 585 16 129 167 296 394 455 355 362 229 779 234 911 480 538 185 681 712 345 809 241 326 832 893 162 391 934 860 357 9 413 295 377 542 462 673 289 917 381 651 498 512 886 409 344 424 947 26 488 644 187 81 971 371 975 133 114 261 345 472 622 759 119 0 653 581 25 942 850 758 946 
Sequential Bubble Sort took : 5.8157e-05 seconds.
0 9 16 23 25 26 40 59 72 75 81 114 119 127 129 133 162 167 173 177 185 187 201 229 234 241 261 265 274 289 295 296 300 304 326 344 345 345 355 357 362 371 377 381 391 394 409 413 422 424 455 462 472 480 488 498 512 538 542 558 575 581 581 585 616 622 629 644 651 653 673 681 681 697 700 712 758 759 779 808 809 832 836 849 850 860 872 886 893 911 917 928 934 936 942 946 947 966 971 975 
Parallel Bubble Sort took : 4.1103e-05 seconds.
0 9 16 23 25 26 40 59 72 75 81 114 119 127 129 133 162 167 173 177